In [331]:
import pandas as pd
import numpy as np

from selenium import webdriver

from datetime import datetime

import time

In [557]:
def parse_wunderground(code, year, month, day, flag_error):
    
    global errors
    
    try:
        driver = webdriver.Chrome()
        driver.get(f'https://www.wunderground.com/history/daily/gb/ulceby/{code}/date/{year}-{month}-{day}')
        time.sleep(3)
    except:
        if flag_error == False:
            errors.append((code, year, month, day))
        print('driver_get_error', code, year, month, day)
        return None
    
    try:
        df = pd.read_html(driver.page_source)[1]
        driver.quit()
    except:
        if flag_error == False:
            errors.append((code, year, month, day))
        print('read_html_error', code, year, month, day)
        driver.quit()
        return None
        
    try:
        df['Date'] = pd.to_datetime(f'{year}/{month}/{day}')
        df = df.loc[:, ['Date', 'Time', 'Temperature', 'Humidity', 'Wind Speed', 'Condition']].reset_index(drop=True)
    except:
        if flag_error == False:
            errors.append((code, year, month, day))
        print('date_error', code, year, month, day)
        return df
    
    df['Location'] = code
    
    return df

In [355]:
# output = parse_wunderground('EGLC', 2042, 4, 19, flag_error=True)
# output

read_html_error


In [22]:
# собираем датафрейм со всеми датами

In [337]:
df = pd.read_csv('elo_ratings_all_teams', parse_dates=['Date'])
df = df.loc[:, df.columns[1:]]
df

,Date,Time,Location,HomeTeam,AwayTeam,Elo_HomeTeam,Elo_AwayTeam
0,2024-04-25,20:00,EGKK,Brighton,Man City,1748.398682,2038.480591
1,2024-04-24,20:00,EGCC,Man United,Sheffield United,1793.209839,1551.766602
2,2024-04-24,20:00,EGGP,Everton,Liverpool,1692.962402,1923.581787
3,2024-04-24,20:00,EGLC,Crystal Palace,Newcastle,1708.831055,1811.576782
4,2024-04-24,19:45,EGBB,Wolves,Bournemouth,1711.067993,1695.033447
...,...,...,...,...,...,...,...
2159,2018-08-11,15:00,EGNT,Newcastle,Tottenham,1670.871338,1914.848877
2160,2018-08-11,15:00,EGNM,Huddersfield,Chelsea,1567.101318,1837.004272
2161,2018-08-11,15:00,EGLC,Fulham,Crystal Palace,1633.799683,1692.951660
2162,2018-08-11,15:00,EGHH,Bournemouth,Cardiff,1673.780518,1576.490356


In [356]:
date_loc_unique = df[['Date', 'Location']].drop_duplicates()
date_loc_unique

,Date,Location
0,2024-04-25,EGKK
1,2024-04-24,EGCC
2,2024-04-24,EGGP
3,2024-04-24,EGLC
4,2024-04-24,EGBB
...,...,...
2159,2018-08-11,EGNT
2160,2018-08-11,EGNM
2161,2018-08-11,EGLC
2162,2018-08-11,EGHH


## Парсер по свежим данным 

#### Уже было сделано по [:1000], около 130 не спарсились
- Прошлись по [:1000]. Первые сохранили в файл [1]parsed_weather_:1000.csv
- Потом прошлись по ошибкам errors. Не обработалось около 130 ошибок все равно. Сохранил результаты в [1]good_errors:1000.csv

#### Сделал [1000:1500]
- Прошелся 500. Оставалось много ошибок
- Прошелся по остающимся ошибкам 4 раза: осталось всего 9 несчитанных таблиц, потому что данные отсутствуют
- Сохранил результаты в [2]good_errors:1000.csv и тд

#### Сделал [1500:]
- Удалось обработать все с 0 ошибок
- Сохранил результаты в файлы [3]good_errors1500.csv и тд

#### Переделал первую 1000, так как нашел метод избавиться от ошибок
- Удалось обработать все с 22 ошибками. Все ошибки были из-за отстутствия данных на страницах


In [288]:
# # Первые [:1000]
# from tqdm import tqdm

# errors = []
# dataframes_to_concat = []

# for row in tqdm(date_loc_unique.values[1000:1010]):
#     result = parse_wunderground(row[1], row[0].year, row[0].month, row[0].day, flag_error=False)
    
#     if isinstance(result, pd.core.frame.DataFrame):
#         dataframes_to_concat.append(result)                


 19%|███████▎                              | 193/1000 [27:04<1:58:35,  8.82s/it]


 39%|██████████████▋                       | 387/1000 [59:08<1:29:36,  8.77s/it]


 58%|██████████████████████                | 581/1000 [1:27:12<55:32,  7.95s/it]


 78%|█████████████████████████████▍        | 775/1000 [1:53:31<29:04,  7.75s/it]


 97%|████████████████████████████████████▊ | 969/1000 [2:20:37<04:12,  8.13s/it]


100%|█████████████████████████████████████| 1000/1000 [2:25:25<00:00,  8.73s/it]


In [464]:
# Проход по 1000:1500
# from tqdm import tqdm

# errors = []
# dataframes_to_concat = []

# for row in tqdm(date_loc_unique.values[1000:1500]):
#     result = parse_wunderground(row[1], row[0].year, row[0].month, row[0].day, flag_error=False)
    
#     if isinstance(result, pd.core.frame.DataFrame):
#         dataframes_to_concat.append(result)                

  1%|▍                                        | 6/500 [01:04<1:29:57, 10.93s/it]

read_html_error EGNT 2021 3 12


  1%|▌                                        | 7/500 [01:11<1:20:08,  9.75s/it]

read_html_error EGCC 2021 3 10


  2%|▋                                        | 8/500 [01:22<1:22:22, 10.05s/it]

...

 99%|██████████████████████████████████████▌| 494/500 [1:16:22<00:48,  8.01s/it]

read_html_error EGGP 2019 11 23


100%|██████████████████████████████████████▉| 499/500 [1:17:05<00:08,  8.37s/it]

read_html_error EGCC 2019 11 10


100%|███████████████████████████████████████| 500/500 [1:17:12<00:00,  9.27s/it]


In [506]:
# # Проход по 1500:
# from tqdm import tqdm

# errors = []
# dataframes_to_concat = []

# for row in tqdm(date_loc_unique.values[1500:]):
#     result = parse_wunderground(row[1], row[0].year, row[0].month, row[0].day, flag_error=False)
    
#     if isinstance(result, pd.core.frame.DataFrame):
#         dataframes_to_concat.append(result)                

  1%|▍                                          | 4/435 [00:20<35:56,  5.00s/it]

read_html_error EGNM 2019 11 9


  1%|▍                                          | 5/435 [00:26<36:32,  5.10s/it]

read_html_error EGSH 2019 11 8


  3%|█                                         | 11/435 [00:58<37:13,  5.27s/it]

...

 99%|████████████████████████████████████████▌| 431/435 [38:03<00:18,  4.64s/it]

read_html_error EGNT 2018 8 11


100%|████████████████████████████████████████▉| 434/435 [38:18<00:04,  4.78s/it]

read_html_error EGHH 2018 8 11


100%|█████████████████████████████████████████| 435/435 [38:23<00:00,  5.30s/it]


In [561]:
# Проход по 0:1000: # TO BE DONE
from tqdm import tqdm

errors = []
dataframes_to_concat = []

for row in tqdm(date_loc_unique.values[:1000]):
    result = parse_wunderground(row[1], row[0].year, row[0].month, row[0].day, flag_error=False)
    
    if isinstance(result, pd.core.frame.DataFrame):
        dataframes_to_concat.append(result)                

  1%|▍                                      | 10/1000 [01:16<2:00:02,  7.27s/it]

read_html_error EGCN 2024 4 20


  2%|▋                                      | 17/1000 [02:14<2:14:35,  8.22s/it]

read_html_error EGNM 2024 4 13


  2%|▊                                      | 20/1000 [05:36<9:23:38, 34.51s/it]

...

 99%|█████████████████████████████████████▋| 993/1000 [2:32:22<01:08,  9.73s/it]

read_html_error EGNM 2021 4 3


100%|█████████████████████████████████████▉| 999/1000 [2:33:10<00:08,  8.17s/it]

read_html_error EGHI 2021 3 14


100%|█████████████████████████████████████| 1000/1000 [2:33:18<00:00,  9.20s/it]


In [562]:
len(errors)

27

In [563]:
weather = pd.concat(dataframes_to_concat)
weather

,Date,Time,Temperature,Humidity,Wind Speed,Condition,Location
0,2024-04-25,12:20 AM,36 °F,81 °%,1 °mph,Fair,EGKK
1,2024-04-25,12:50 AM,36 °F,87 °%,1 °mph,Fair,EGKK
2,2024-04-25,1:20 AM,34 °F,87 °%,1 °mph,Fair,EGKK
3,2024-04-25,1:50 AM,36 °F,87 °%,1 °mph,Fair,EGKK
4,2024-04-25,2:20 AM,36 °F,87 °%,1 °mph,Fair,EGKK
...,...,...,...,...,...,...,...
53,2021-03-14,NaN,NaN,NaN,NaN,NaN,EGCC
54,2021-03-14,NaN,NaN,NaN,NaN,NaN,EGCC
55,2021-03-14,NaN,NaN,NaN,NaN,NaN,EGCC
56,2021-03-14,NaN,NaN,NaN,NaN,NaN,EGCC


In [564]:
parsed_weather.append(weather)

## В конце: сохраним данные о погоде, которые уже спарсили

In [565]:
weather.to_csv('[4]parsed_weather_:1000.csv')

## Обработка страниц, где вылезла ошибка с чтением таблиц

In [566]:
errs = errors.copy()

In [575]:
dataframes_to_concat = []
bads = []

for code, year, month, day in tqdm(errs):
    result = parse_wunderground(code, year, month, day, flag_error=True)
    
    if isinstance(result, pd.core.frame.DataFrame):
        dataframes_to_concat.append(result)
    else:
        bads.append((code, year, month, day))

  5%|██                                          | 1/22 [00:12<04:15, 12.14s/it]

read_html_error EGCN 2024 4 20


  9%|████                                        | 2/22 [00:20<03:17,  9.87s/it]

read_html_error EGCN 2024 4 7


 14%|██████                                      | 3/22 [00:27<02:43,  8.63s/it]

...

read_html_error EGHI 2021 4 4


 95%|█████████████████████████████████████████  | 21/22 [03:13<00:08,  8.07s/it]

read_html_error EGNM 2021 4 3


100%|███████████████████████████████████████████| 22/22 [03:22<00:00,  9.21s/it]

read_html_error EGHI 2021 3 14


In [576]:
len(bads)

22

In [577]:
errs = bads.copy()
errs

[('EGCN', 2024, 4, 20),
 ('EGCN', 2024, 4, 7),
 ('EGCN', 2024, 3, 30),
 ('EGCN', 2024, 3, 4),
 ('EGCN', 2024, 2, 18),
 ('EGCN', 2024, 2, 3),
 ('EGCN', 2024, 1, 21),
 ('EGCN', 2023, 12, 26),
 ('EGCN', 2023, 12, 9),
 ('EGCN', 2023, 12, 6),
 ('EGCN', 2023, 11, 25),
 ('EGCN', 2023, 11, 4),
 ('EGCN', 2023, 10, 21),
 ('EGCN', 2023, 9, 24),
 ('EGCN', 2023, 9, 2),
 ('EGCN', 2023, 8, 27),
 ('EGCN', 2023, 8, 12),
 ('EGLC', 2022, 1, 1),
 ('EGNM', 2021, 4, 11),
 ('EGHI', 2021, 4, 4),
 ('EGNM', 2021, 4, 3),
 ('EGHI', 2021, 3, 14)]

In [578]:
len(dataframes_to_concat)

0

In [572]:
good_errors = pd.concat(dataframes_to_concat)
good_errors

,Date,Time,Temperature,Humidity,Wind Speed,Condition,Location
0,2024-04-13,12:20 AM,52 °F,76 °%,13 °mph,Mostly Cloudy,EGNM
1,2024-04-13,12:50 AM,52 °F,82 °%,13 °mph,Mostly Cloudy,EGNM
2,2024-04-13,1:20 AM,52 °F,82 °%,12 °mph,Mostly Cloudy,EGNM
3,2024-04-13,1:50 AM,52 °F,82 °%,9 °mph,Mostly Cloudy,EGNM
4,2024-04-13,2:20 AM,52 °F,82 °%,10 °mph,Mostly Cloudy,EGNM
...,...,...,...,...,...,...,...
53,2021-08-28,NaN,NaN,NaN,NaN,NaN,EGCC
54,2021-08-28,NaN,NaN,NaN,NaN,NaN,EGCC
55,2021-08-28,NaN,NaN,NaN,NaN,NaN,EGCC
56,2021-08-28,NaN,NaN,NaN,NaN,NaN,EGCC


In [573]:
good_errors.to_csv('[4]good_errors_1_1000:.csv')

In [574]:
parsed_weather.append(good_errors)
parsed_weather

[         Date   Time Temperature Humidity Wind Speed      Condition Location
 0  2024-04-25  12:20       52 °F    50 °%    14 °mph           Fair     EGKK
 1  2024-04-25  12:50       54 °F    47 °%    15 °mph           Fair     EGKK
 2  2024-04-25  13:20       54 °F    51 °%    14 °mph           Fair     EGKK
 3  2024-04-25  13:50       52 °F    54 °%    12 °mph  Mostly Cloudy     EGKK
 4  2024-04-25  14:20       54 °F    51 °%     9 °mph           Fair     EGKK
 ..        ...    ...         ...      ...        ...            ...      ...
 14 2021-05-13  19:20       54 °F    88 °%     7 °mph           Fair     EGCC
 15 2021-05-13  19:50       55 °F    77 °%     8 °mph           Fair     EGCC
 16 2021-05-13  20:20       55 °F    67 °%     9 °mph           Fair     EGCC
 17 2021-05-13  20:50       54 °F    72 °%     9 °mph  Mostly Cloudy     EGCC
 18 2021-05-13  21:20       52 °F    76 °%     8 °mph           Fair     EGCC
 
 [11552 rows x 7 columns],
          Date      Time Temperatur

### Точечно смотрим на ошибки

In [463]:
code, year, month, day = ('EGNM', 2021, 3, 13)

res = parse_wunderground(code, year, month, day, flag_error=True)
res

#code, year, month, day = ('EGGP', 2024, 4, 24)
#code, year, month, day = ('EGHH', 2024, 4, 13)

#f'https://www.wunderground.com/history/daily/gb/ulceby/{code}/date/{year}-{month}-{day}'

,Date,Time,Temperature,Humidity,Wind Speed,Condition,Location
0,2021-03-13,1:20 PM,41 °F,81 °%,24 °mph,Partly Cloudy / Windy,EGNM
1,2021-03-13,1:50 PM,41 °F,81 °%,26 °mph,Light Rain Shower / Windy,EGNM
2,2021-03-13,2:20 PM,43 °F,76 °%,29 °mph,Partly Cloudy / Windy,EGNM
3,2021-03-13,2:50 PM,39 °F,87 °%,21 °mph,Partly Cloudy / Windy,EGNM
4,2021-03-13,3:20 PM,41 °F,81 °%,26 °mph,Partly Cloudy / Windy,EGNM
5,2021-03-13,3:50 PM,41 °F,76 °%,24 °mph,Partly Cloudy / Windy,EGNM
6,2021-03-13,4:20 PM,43 °F,70 °%,22 °mph,Partly Cloudy / Windy,EGNM
7,2021-03-13,4:50 PM,41 °F,81 °%,21 °mph,Partly Cloudy / Windy,EGNM
8,2021-03-13,5:20 PM,41 °F,76 °%,21 °mph,Partly Cloudy / Windy,EGNM
9,2021-03-13,5:50 PM,41 °F,76 °%,17 °mph,Partly Cloudy,EGNM


# Важно! Обработка AM PM. Нужно провести.
Будут ошибки, их надо обработать отдельно

In [400]:
res['Time'] = res['Time'].apply(lambda x: x if x is None else datetime.strptime(x, '%I:%M %p').strftime('%H:%M'))

ValueError: time data '13:20' does not match format '%I:%M %p'